In [1]:
import math
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd
from matplotlib.mlab import PCA as mlabPCA
from sklearn import linear_model
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

def my_plot_loop(X,y):
    
    #takes an X feature set (df or array)
    #takes a y expected outcome set (series, list, or array)
    #plots all features (histogram, boxplot, and scatter against y
    
    list_features = X.columns
    listlength = len(list_features)+1
    f, axes = plt.subplots(listlength, 3, sharey=False, sharex=False , figsize=(20,20))
    y_var = y.name
    
    i = 1
    for column in list_features:
        plt.subplot(listlength, 3, i)
        sns.scatterplot(X[column],y)

        plt.subplot(listlength, 3, i+1)
        sns.boxplot(X[column] )

        plt.subplot(listlength, 3, i+2)
        #plt.title(column)
        plt.xlabel(column)
        plt.hist(X[column], bins="auto")
        i = i + 3

    #Plot Y Variable
    plt.subplot(listlength, 3, i)
    sns.scatterplot(y,y)

    plt.subplot(listlength, 3, i+1)
    sns.boxplot(y )

    plt.subplot(listlength, 3, i+2)
    #plt.title(column)
    plt.xlabel(y_var)
    plt.hist(y, bins="auto")
    return None
    

In [2]:
def my_pre_processing(raw, pop_cutoff):

    raw.columns = ['City','Population','Violent_Crime','Murder','Rape1','Rape2','Robbery','Assault',
                   'Property_Crime','Burglary','Larceny_Theft','Vehicle_Theft','Arson']
    df = pd.DataFrame(raw)


    #Data Cleaning
    df.at[df['Rape2'].isna() , 'Rape2'] =  df['Rape1']  
    df.drop(columns='Rape1', axis=1, inplace=True)
    df.dropna(subset=['Population'],inplace=True)
    df['Arson'] = np.where(df.Arson.isnull(),0,1)
    df = df[df.Population < pop_cutoff]

    #Feature Engineering
    df['binaryMurder'] = np.where(df.Murder > 0, 1, 0)
    df['binaryRobbery'] = np.where(df.Robbery > 0, 1, 0)
    df['binaryAssault'] = np.where(df.Assault > 0, 1, 0)
    df['binaryRape'] = np.where(df.Rape2 > 0, 1, 0)
    
    df['Murder_Sqrt'] = np.sqrt(df.Murder)
    df['Violent_Crime_Sqrt'] = df.Violent_Crime ** (1/2)
    df['Population_Square'] = np.square(df.Population)
    df['Population_Sqrt'] = np.sqrt(df.Population)
    df['Robbery_Sqrt'] = np.sqrt(df.Robbery)
    df['Assault_Sqrt'] = np.sqrt(df.Assault)
    df['Property_Crime_Sqrt'] = np.sqrt(df.Property_Crime)
    df['Property_Crime_Square'] = np.square(df.Property_Crime)
    df['Rape2_Sqrt'] = np.sqrt(df.Rape2)
    df['Robbery_Assault_Sum'] = df.Assault + df.Robbery
   
    df['RobberyPerHundred'] = df['Robbery'] / df['Population'] * 100
    df['AssaultPerHundred'] = df['Assault'] / df['Population'] * 100

    df['pop_size'] = pd.cut(raw.Population, [0, 3000, 15000, 75000, 200000, 1000000, 99999999999],  
                          labels=["verysmall",'small',"medium","large",'verylarge',"mill"])
    pop_dummies = pd.get_dummies(df['pop_size'])
    df = pd.merge(df, pop_dummies, left_index=True, right_index=True, how='inner')

    df['Violent_Crime'] = pd.cut(raw.Population, bins=3,  
                          labels=["violent_crime_low",'violent_crime_med',"violent_crime_high"])
    pop_dummies = pd.get_dummies(df['Violent_Crime'])
    df = pd.merge(df, pop_dummies, left_index=True, right_index=True, how='inner')
    
    df['Property_Crime_Per_Capita'] = df.Population / df.Property_Crime

    df['Property_Crime_Rate'] = pd.qcut(df.Property_Crime_Per_Capita, q=2, labels=[0,1])
    
    
    feature_list = ['binaryMurder','binaryRobbery','binaryAssault','binaryRape',
                    'verysmall','small','medium','large','verylarge','mill',
                   "violent_crime_low",'violent_crime_med',"violent_crime_high"]
    
    y_var = 'Property_Crime_Rate'
    

    print('Cities processing: {}'.format(df.shape[0]))
    #print('Cities Removed due to pop above {}: '.format(pop_cutoff))
    pop_remove_list = raw.loc[raw.Population > pop_cutoff,['City','Population']].sort_values(
        'Population', ascending=False)
    #if pop_remove_list.shape[0] == 0:
        #print('None')
    #else:
        #print(pop_remove_list)
    
  
    print('\nCities removed due to Null: ')
    null_list = df.isnull().any(axis=1)
    null_list = null_list[null_list.values==True]
    print(len(null_list))
    
    #print(df[df.isnull().any(axis=1)].index)  #,['City','Population']])
    df.dropna(inplace=True)

    X = df[feature_list]
    Y = df[y_var]
    #sns.boxplot(df.Population)
    return X, Y


def my_run_logistic_regr_vanilla(X,Y):

    # Declare predictors.
    X_statsmod = X
   
    # The Statsmodels formulation requires a column with constant value 1 that
    # will act as the intercept.
    X_statsmod['intercept'] = 1 

    
    # Declare and fit the model.
    logit = sm.Logit(Y, X_statsmod, missing='drop')
    result = logit.fit()

    # Lots of information about the model and its coefficients, but the
    # accuracy rate for predictions is missing.
    print(result.summary())
    return result

In [3]:
file1 = 'table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls'
file2 = 'table_8_offenses_known_to_law_enforcement_georgia_by_city_2013.xls'
file3 = 'table_8_offenses_known_to_law_enforcement_illinois_by_city_2013.xls'
file4 = 'table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls'
file5 = 'table_8_offenses_known_to_law_enforcement_utah_by_city_2013.xls'
file6 = 'table_8_offenses_known_to_law_enforcement_kentucky_by_city_2013.xls'
file7 = 'table_8_offenses_known_to_law_enforcement_colorado_by_city_2013.xls'
file8 = 'table_8_offenses_known_to_law_enforcement_alabama_by_city_2013.xls'
file9 = 'table_8_offenses_known_to_law_enforcement_washington_by_city_2013.xls'
file10 = 'table_8_offenses_known_to_law_enforcement_ohio_by_city_2013.xls'

train_files = [file1, file2, file3, file4, file5, file6, file7]
test_files = [file1, file2, file3, file8, file9, file10 ]
pop_cutoff = 999999999

colnames = ['City','Population','Violent_Crime','Murder','Rape1','Rape2','Robbery','Assault',
                   'Property_Crime','Burglary','Larceny_Theft','Vehicle_Theft','Arson']

raw = pd.DataFrame(columns=colnames)

print('----------------------------------------------------')
print('--------------------- TRAINING ---------------------')
print('----------------------------------------------------')
print('------- Pre-Process Training Data -----------')
for file in train_files:
    #print('file_train: {}'.format(file))   
    raw = pd.concat([raw, pd.read_excel(file, skiprows=4, names=colnames, usecols=12)],ignore_index=True )

train_X, train_Y = my_pre_processing(raw, pop_cutoff = pop_cutoff)

print(train_Y.value_counts())


print('\n--------- Create Vanilla Regression Model -----------')
vlr = my_run_logistic_regr_vanilla(train_X, train_Y)




print('\n--------- Create Logistic Regression Model using SciKit default L2 -----------')
lr = linear_model.LogisticRegression(penalty='l2')
lr.fit(train_X, train_Y)

print('R Squared: {}'.format(lr.score(train_X,train_Y)))
print('Intercept ' + str(lr.intercept_))
print(*list(lr.coef_), sep="\n")
cv=10
score = cross_val_score(lr, train_X, train_Y, cv=cv)
print("\nCross Validation Accuracy %i folds: %.2f (+/- %.2f)" % (cv, score.mean(), (score.std() * 2)))



print('\n--------- Create Logistic Regression Model using SciKit L1 -----------')
lr = linear_model.LogisticRegression(penalty='l1')
lr.fit(train_X, train_Y)

print('R Squared: {}'.format(lr.score(train_X,train_Y)))
print('Intercept ' + str(lr.intercept_))
print(*list(lr.coef_), sep="\n")
cv=10
score = cross_val_score(lr, train_X, train_Y, cv=cv)
print("\nCross Validation Accuracy %i folds: %.2f (+/- %.2f)" % (cv, score.mean(), (score.std() * 2)))




print('\n--------- Create Ridge Regression Model -----------')
ridgeregr = linear_model.Ridge(alpha=.5, fit_intercept=False) 
ridgeregr.fit(train_X, train_Y)

print('R Squared: {}'.format(ridgeregr.score(train_X,train_Y)))
print('Intercept ' + str(ridgeregr.intercept_))
print(*list(ridgeregr.coef_), sep="\n")

cv=10
score = cross_val_score(ridgeregr, train_X, train_Y, cv=cv)
print("\nCross Validation Accuracy %i folds: %.2f (+/- %.2f)" % (cv, score.mean(), (score.std() * 2)))






print('\n--------- Create Lasso Regression Model -----------')
lass = linear_model.Lasso(alpha=.0005)
lass.fit(train_X, train_Y)

print('R Squared: {}'.format(lass.score(train_X,train_Y)))
print('Intercept ' + str(lass.intercept_))
print(*list(lass.coef_), sep='\n')

cv=10
score = cross_val_score(lass, train_X, train_Y, cv=cv)
print("\nCross Validation Accuracy %i folds: %.2f (+/- %.2f)" % (cv, score.mean(), (score.std() * 2)))


----------------------------------------------------
--------------------- TRAINING ---------------------
----------------------------------------------------
------- Pre-Process Training Data -----------
Cities processing: 1989

Cities removed due to Null: 
3
1    993
0    993
Name: Property_Crime_Rate, dtype: int64

--------- Create Vanilla Regression Model -----------
         Current function value: 0.560412
         Iterations: 35
                            Logit Regression Results                           
Dep. Variable:     Property_Crime_Rate   No. Observations:                 1986
Model:                           Logit   Df Residuals:                     1974
Method:                            MLE   Df Model:                           11
Date:                 Mon, 22 Oct 2018   Pseudo R-squ.:                  0.1915
Time:                         15:15:58   Log-Likelihood:                -1113.0
converged:                       False   LL-Null:                       -1376.6


/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)



Cross Validation Accuracy 10 folds: 0.70 (+/- 0.11)

--------- Create Ridge Regression Model -----------
R Squared: 0.24051285510630804
Intercept 0.0
-0.1446130982864229
-0.3446229692815565
-0.19234577637488867
-0.20298127911337582
-0.023659710851359497
0.09313707239922892
0.26288383910016166
0.14193241751635333
0.11717692579947052
0.1717076257753402
0.1636083996824921
-0.03354844830526835
0.6331182183613984
0.7631781697388063

Cross Validation Accuracy 10 folds: 0.17 (+/- 0.21)

--------- Create Lasso Regression Model -----------
R Squared: 0.2392002896250187
Intercept 1.0454944777426143
-0.13921847140604682
-0.3409687373589269
-0.18877742064386482
-0.19948005861635076
-0.1439721132602608
-0.032596789821763436
0.1302706820237793
0.0013785889543238424
-0.0
0.0
-0.0
-0.0
0.0
0.0

Cross Validation Accuracy 10 folds: 0.17 (+/- 0.21)
